In [ ]:
import requests
import csv
import json

# Replace with your GitHub Personal Access Token
GITHUB_TOKEN = 'YOUR_PERSONAL_ACCESS_TOKEN'

# Function to fetch issues for a repository
def get_issues(owner, repo):
    url = f'https://api.github.com/repos/{owner}/{repo}/issues'
    response = requests.get(url, headers={'Authorization': f'token {GITHUB_TOKEN}'})
    if response.status_code == 200:
        return response.json()
    else:
        return []

# Function to fetch comments for an issue
def get_comments(owner, repo, issue_number):
    url = f'https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/comments'
    response = requests.get(url, headers={'Authorization': f'token {GITHUB_TOKEN}'})
    if response.status_code == 200:
        return response.json()
    else:
        return []

# Function to filter issues with at least 1 comment
def filter_issues_with_comments(issues):
    return [issue for issue in issues if issue['comments'] >= 1]

# Read the CSV file
with open('repository_list.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Create CSV files for issues with comments and releases
    with open('filtered_issues.csv', 'w', newline='') as filtered_issues_file:
        filtered_issues_writer = csv.writer(filtered_issues_file)
        filtered_issues_writer.writerow(['Repository', 'Issue Title', 'Issue URL', 'Comment Count'])

        with open('releases_new.csv', 'w', newline='') as releases_file:
            releases_writer = csv.writer(releases_file)
            releases_writer.writerow(['Repository', 'Release Name', 'Release URL'])

            for row in csv_reader:
                owner = row['owner']
                repo = row['repo']

                # Fetch issues for the repository
                issues = get_issues(owner, repo)

                # Fetch releases for the repository
                releases = get_releases(owner, repo)

                # Filter issues with at least 1 comment
                filtered_issues = filter_issues_with_comments(issues)

                # Write filtered issues with comment count to the CSV file
                for issue in filtered_issues:
                    issue_number = issue['number']
                    comments = get_comments(owner, repo, issue_number)
                    comment_count = len(comments)
                    filtered_issues_writer.writerow([f'{owner}/{repo}', issue['title'], issue['html_url'], comment_count])

                # Write releases to the CSV file
                for release in releases:
                    releases_writer.writerow([f'{owner}/{repo}', release['name'], release['html_url']])
